In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.seqsignet_functions import (
    seqsignet_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
os.chdir("../../")

Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

Seq-Sig-Net

In [7]:
time_features = ["time_encoding"]
standardise_method = ["standardise"]
num_time_features = len(time_features)
add_time_in_path = False

In [8]:
num_epochs = 100
batch=64
embedding_dim = sbert_embeddings.shape[1]
dim_reduce_method = ["umap"]
dimensions = [15] 
conv_output_channels = [10]
swnu_hidden_dim_sizes_and_sig_depths = [([10], 3)]
bidirectional = False
lstm_hidden_dim_sizes = [250, 350]
ffn_hidden_dim_sizes = [[64,64],[128,128],[256,256]] 
dropout_rates = [0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [9]:
shift = 3
window_size = 5
n = 11

In [10]:
size = shift*n+(window_size-shift)
print(size)

seqsignet_network_umap, best_seqsignet_network_umap, _, __ = seqsignet_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    embedding_dim=embedding_dim,
    output_dim=output_dim,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=dim_reduce_method,
    dimensions=dimensions,
    log_signature=True,
    conv_output_channels=conv_output_channels,
    swnu_hidden_dim_sizes_and_sig_depths=swnu_hidden_dim_sizes_and_sig_depths,
    lstm_hidden_dim_sizes=lstm_hidden_dim_sizes,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    BiLSTM=bidirectional,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    time_feature=time_features,
    standardise_method=standardise_method,
    add_time_in_path=add_time_in_path,
    path_indices=None,
    #data_split_seed=0,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

35


  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.







































































































































































































































100%|██████████| 5568/5568 [00:25<00:00, 220.43it/s]


[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.









































100%|██████████| 3/3 [1:50:36<00:00, 2212.02s/it]





























100%|██████████| 3/3 [1:46:51<00:00, 2137.03s/it]





100%|██████████| 2/2 [3:37:27<00:00, 6523.57s/it]




100%|██████████| 1/1 [3:37:27<00:00, 13047.14s/it]




































100%|██████████| 3/3 [2:20:19<00:00, 2806.36s/it]





























100%|██████████| 3/3 [2:25:46<00:00, 2915.35s/it]





100%|██████████| 2/2 [4:46:05<00:00, 8582.57s/it]




100%|██████████| 1/1 [4:46:05<00:00, 17165.14s/it]


























: 

: 

In [11]:
seqsignet_network_umap

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,k,shift,...,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,0.246717,0.609152,0.587372,"[0.6821705426356589, 0.4925742574257425]",0.247113,0.601424,0.517833,"[0.7185929648241206, 0.31707317073170727]",20,3,...,False,focal,2,False,None,Conv1d,None,concatenation,64,0
0,0.238242,0.610105,0.607824,"[0.5779153766769866, 0.6377325066430468]",0.237390,0.594306,0.509223,"[0.71356783919598, 0.30487804878048774]",20,3,...,False,focal,2,False,None,Conv1d,None,concatenation,64,0


In [ ]:
best_seqsignet_network_umap

In [ ]:
best_seqsignet_network_umap["valid_f1"].mean()

In [12]:
best_seqsignet_network_umap["f1"].mean()

0.5975981708453586

In [ ]:
best_seqsignet_network_umap["precision"].mean()

In [ ]:
best_seqsignet_network_umap["recall"].mean()

In [14]:
np.stack(best_seqsignet_network_umap["f1_scores"]).mean(axis=0)

array([0.63004296, 0.56515338])